<a href="https://colab.research.google.com/github/adilabduakhanov/Developing_Neuro-Employees_on_GPT/blob/main/Fine_tuning_chatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ДЗ Лайт

Используйте матриалы теоритеского и практического ноутбуков и **составьте датасет из не менее 10 диалогов** (или вопрос-ответ, или же многоходовый диалог) и обучите модель **в своем стиле** (*без подробной инструкции, а на примерах в своем уникальном стиле: стилизации вывода, транформации текста, разговора на вымышленном языке или любым другим способом*) отвечать на вопросы.

**Этот инструмент** может помочть **Вам** создать небольшой датасет вопрос/ответ для различных задач конвертируя его сразу в JSONL формат для fine tuning chatgpt.

**Add Row** - **добавляет строку в датафрейм** с данными заполненными полей System, User и Assistant.

**После заполнения** необходимого количества примеров, нажмите на кнопку Save CSV and JSONL.

Таким образом создатся файл **fine_tune_dataset.jsonl** и **data.csv**.

In [ ]:
#@title Интерактивный инструмент создания csv таблицы из своих примеров

import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
import json

# Создаем пустой DataFrame
df = pd.DataFrame(columns=['system', 'user', 'assistant'])

# Функция для добавления строки в DataFrame
def add_row(system, user, assistant):
    global df
    new_row = pd.DataFrame({'system': [system], 'user': [user], 'assistant': [assistant]})
    df = pd.concat([df, new_row], ignore_index=True)
    update_display()

# Функция для обновления отображения DataFrame и счетчика
def update_display():
    styles = [
        {'selector': 'th', 'props': [('border', '2px solid black')]},  # Границы для заголовков
        {'selector': 'td', 'props': [('border', '2px solid black')]},  # Границы для ячеек
        {'selector': 'th.col0', 'props': [('min-width', '50px'), ('max-width', '50px')]},  # Ширина колонки с индексами
        {'selector': 'th.col1', 'props': [('min-width', '150px'), ('max-width', '150px')]},  # Ширина колонки 'system'
        {'selector': 'th.col2', 'props': [('min-width', '150px'), ('max-width', '150px')]},  # Ширина колонки 'user'
        {'selector': 'th.col3', 'props': [('min-width', '150px'), ('max-width', '150px')]}  # Ширина колонки 'assistant'
    ]
    df_styled = df.style.set_table_styles(styles).set_properties(**{
        'border': '2px solid black',
        'min-width': '500px',
        'max-width': '500px'
    })
    df_display.value = df_styled.to_html(index=True)
    count_label.value = f"Количество строк: {len(df)}"

# Виджеты для ввода данных с увеличенной шириной
input_layout = widgets.Layout(width='1200px')
system_input = widgets.Text(description='System:', layout=input_layout)
user_input = widgets.Text(description='User:', layout=input_layout)
assistant_input = widgets.Text(description='Assistant:', layout=input_layout)
add_button = widgets.Button(description='Add Row', layout=widgets.Layout(width='200px'))
save_button = widgets.Button(description='Save CSV and JSONL', layout=widgets.Layout(width='200px'))

# Виджеты для отображения счетчика и DataFrame
count_label = widgets.Label(value="Количество строк: 0")
df_display = widgets.HTML(value=df.to_html(index=False))

# Обработчики событий
def on_add_button_clicked(b):
    add_row(system_input.value, user_input.value, assistant_input.value)
    system_input.value = ''
    user_input.value = ''
    assistant_input.value = ''

def on_save_button_clicked(b):
    df.to_csv('data.csv', index=False)
    print("CSV файл сохранен")

    fine_tune_data = []
    for index, row in df.iterrows():
      user = row['user']
      assistant = row['assistant']
      fine_tune_data.append({
          "messages": [
              {"role": "system", "content": row['system']},
              {"role": "user", "content": user},
              {"role": "assistant", "content": assistant}
          ]
      })

    # Save the dataset in JSONL format
    dataset_path = 'fine_tune_dataset.jsonl'
    with open(dataset_path, 'w') as f:
        for item in fine_tune_data:
            f.write(json.dumps(item) + '\n')

    print(f"Датасет подготовлен и сохранен для fine-tuning: {dataset_path}")

# Привязка обработчиков к кнопкам
add_button.on_click(on_add_button_clicked)
save_button.on_click(on_save_button_clicked)

# Отображение виджетов
display(system_input, user_input, assistant_input, add_button, save_button, count_label, df_display)


Text(value='', description='System:', layout=Layout(width='1200px'))

Text(value='', description='User:', layout=Layout(width='1200px'))

Text(value='', description='Assistant:', layout=Layout(width='1200px'))

Button(description='Add Row', layout=Layout(width='200px'), style=ButtonStyle())

Button(description='Save CSV and JSONL', layout=Layout(width='200px'), style=ButtonStyle())

Label(value='Количество строк: 0')

HTML(value='<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>sy…

CSV файл сохранен
Датасет подготовлен и сохранен для fine-tuning: fine_tune_dataset.jsonl


In [ ]:
!pip install openai


In [ ]:
from google.colab import userdata
from openai import OpenAI

client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"))


In [ ]:
file = client.files.create(
    file=open("fine_tune_dataset.jsonl", "rb"),
    purpose="fine-tune"
)

file.id


'file-WAvAZsz2gctBGfMBcogK7z'

In [ ]:
job = client.fine_tuning.jobs.create(
    training_file=file.id,
    model="gpt-3.5-turbo"
)

job


FineTuningJob(id='ftjob-bWCE7XXMUmOma7odj604sO5w', created_at=1771415576, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-GCjjbIQEyKv4eNzgtd8qulWk', result_files=[], seed=1598340048, status='validating_files', trained_tokens=None, training_file='file-WAvAZsz2gctBGfMBcogK7z', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'))), user_provided_suffix=None, usage_metrics=None, shared_with_openai=False, eval_id=None, internal_worker_backend=None)

In [ ]:
client.fine_tuning.jobs.list()


SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-bWCE7XXMUmOma7odj604sO5w', created_at=1771415576, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=10), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-GCjjbIQEyKv4eNzgtd8qulWk', result_files=[], seed=1598340048, status='validating_files', trained_tokens=None, training_file='file-WAvAZsz2gctBGfMBcogK7z', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=10))), user_provided_suffix=None, usage_metrics=None, shared_with_openai=False, eval_id=None, internal_worker_backend=None), FineTuningJob(id='ftjob-SPHR0xDBijtQTYujxzF2n8GK', created_at=1771412986, error=Error(code=N

In [ ]:
fine_model = "ft:gpt-3.5-turbo-0125:personal::DAZoP3YE"

response = client.responses.create(
    model=fine_model,
    input=[
        {
            "role": "system",
            "content": "Ты отвечаешь в моем уникальном стиле."
        },
        {
            "role": "user",
            "content": "Дай совет по продуктивности."
        }
    ]
)

print(response.output_text)



Не тяни кота за хвост — начни с самого простого. Сделай список задач, раздроби их на глотки, и грызи по одной.


Цель:

Обучить модель отвечать в авторском стиле через fine-tuning.

Что сделано:

Создан датасет из 10+ диалогов

Подготовлен JSONL

Запущен fine-tuning на gpt-3.5-turbo-0125

Получена персональная модель

Протестирована на новом вопросе

Результат:

Модель успешно воспроизводит авторский стиль ответа.